In [3]:
from server_comms import ServerComms, ServerMessageTypes
import time
import threading
import json
from IPython.display import clear_output

In [4]:
server_message_types = ServerMessageTypes()

In [5]:
class ApiRunner:
    def __init__(self):
        self.is_connected = True
        self.server = ServerComms("127.0.0.1", 8052)
            
    def send(self, type, payload=None):
        self.server.send_message(type, payload)
        
    def disconnect(self):
        self.is_connected = False
        self.server.server_socket.close()

In [6]:
api_runner = ApiRunner()

In [12]:
class TankRunner:
    def __init__(self, name, api_runner):
        self.name = name
        self.api_runner = api_runner
        self.lock = threading.Lock()
        self.start()

    def start(self):
        self.has_ended = False
        self.state = dict(me={}, enemies={}, items={})
        self.spawn()
        self.receive_thread = threading.Thread(target=self.read)
        self.receive_thread.daemon = True
        self.receive_thread.start()
        
    def spawn(self):
        self.send(server_message_types.CREATETANK, dict(Name=self.name))
        
    def despawn(self):
        self.send(server_message_types.DESPAWNTANK)
        
    def read(self):
        while not self.has_ended:
            self.lock.acquire()
            message = self.api_runner.server.read_message()
            self.update_state(message)
            print(self.state)
            self.lock.release()
        print("phew finished")
    
    def update_state(self, message):
        if message['type'] == server_message_types.OBJECTUPDATE:
            data = message['data']
            if data['Type'] == 'Tank':
                if data['Name'] == self.name:
                    self.update_me(data)
                elif data['Name'] != self.name:
                    self.update_enemy(data)
            elif data['Type'] == 'AmmoPickup':
                self.update_item(data)
    
    def update_me(self, data):
        self.state['me'] = data
    
    def update_enemy(self, data):
        self.state['enemies'][data['Name']] = data
    
    def update_item(self, data):
        self.state['items'][data['Id']] = data
    
    def send(self, type, payload=None):
        self.lock.acquire()
        self.api_runner.send(type, payload)
        self.lock.release()
    
    def stop(self):
        self.has_ended = True
        self.despawn()
        
    def action(self, action_type):
        if action_type == 1:
            self.send(server_message_types.TOGGLEFORWARD)
        elif action_type == 2:
            self.send(server_message_types.TOGGLEREVERSE)
        elif action_type == 3:
            self.send(server_message_types.TOGGLELEFT)
        elif action_type == 4:
            self.send(server_message_types.TOGGLERIGHT)
        elif action_type == 5:
            self.send(server_message_types.TOGGLETURRETLEFT)
        elif action_type == 6:
            self.send(server_message_types.TOGGLETURRETRIGHT)
            
    def calculate_reward(self):
        pass
    
    def get_state_details(self):
        pass
    
    def step(self, action):
        state = self.get_state_details()
        reward = self.calculate_reward()
        return self.state, reward

In [9]:
for i in range(2):
    tank_runner = TankRunner("bruh{0}".format(i), api_runner)
    time.sleep(5)
    tank_runner.stop()

{'me': {}, 'enemies': {}, 'items': {-332: {'Id': -332, 'Name': '', 'Type': 'AmmoPickup', 'X': -13.373024940490723, 'Y': -83.5798568725586, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}}}
{'me': {}, 'enemies': {}, 'items': {-332: {'Id': -332, 'Name': '', 'Type': 'AmmoPickup', 'X': -13.373024940490723, 'Y': -83.5798568725586, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}}}
{'me': {}, 'enemies': {}, 'items': {-332: {'Id': -332, 'Name': '', 'Type': 'AmmoPickup', 'X': -13.373024940490723, 'Y': -83.5798568725586, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}}}
{'me': {}, 'enemies': {}, 'items': {-332: {'Id': -332, 'Name': '', 'Type': 'AmmoPickup', 'X': -13.373024940490723, 'Y': -83.5798568725586, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}}}
{'me': {}, 'enemies': {}, 'items': {-332: {'Id': -332, 'Name': '', 'Type': 'AmmoPickup', 'X': -13.373024940490723, 'Y': -83.5798568725586, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0

{'me': {'Id': -2234, 'Name': 'bruh0', 'Type': 'Tank', 'X': 69.01915740966797, 'Y': 3.5943377017974854, 'Heading': 300.0, 'TurretHeading': 300.0, 'Health': 10, 'Ammo': 10}, 'enemies': {'ManualTank': {'Id': -344, 'Name': 'ManualTank', 'Type': 'Tank', 'X': 62.115997314453125, 'Y': 46.24128341674805, 'Heading': 343.8318786621094, 'TurretHeading': 343.8318786621094, 'Health': -1, 'Ammo': 10}}, 'items': {-332: {'Id': -332, 'Name': '', 'Type': 'AmmoPickup', 'X': -13.373024940490723, 'Y': -83.5798568725586, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}}}
{'me': {'Id': -2234, 'Name': 'bruh0', 'Type': 'Tank', 'X': 69.01915740966797, 'Y': 3.5943377017974854, 'Heading': 300.0, 'TurretHeading': 300.0, 'Health': 10, 'Ammo': 10}, 'enemies': {'ManualTank': {'Id': -344, 'Name': 'ManualTank', 'Type': 'Tank', 'X': 62.115997314453125, 'Y': 46.24128341674805, 'Heading': 343.8318786621094, 'TurretHeading': 343.8318786621094, 'Health': -1, 'Ammo': 10}}, 'items': {-332: {'Id': -332, 'Name': ''

{'me': {'Id': -2234, 'Name': 'bruh0', 'Type': 'Tank', 'X': 71.22640991210938, 'Y': -5.88977575302124, 'Heading': 112.39608001708984, 'TurretHeading': 112.39608001708984, 'Health': 9, 'Ammo': 10}, 'enemies': {'ManualTank': {'Id': -344, 'Name': 'ManualTank', 'Type': 'Tank', 'X': 51.904788970947266, 'Y': 44.14523696899414, 'Heading': 7.673296928405762, 'TurretHeading': 7.673288345336914, 'Health': -1, 'Ammo': 10}, 'AITank1': {'Id': -512, 'Name': 'AITank1', 'Type': 'Tank', 'X': 57.81350326538086, 'Y': 22.36248779296875, 'Heading': 61.344879150390625, 'TurretHeading': 61.48875045776367, 'Health': 8, 'Ammo': 10}, 'AITank2': {'Id': -680, 'Name': 'AITank2', 'Type': 'Tank', 'X': 45.51226043701172, 'Y': 69.1652603149414, 'Heading': 185.48715209960938, 'TurretHeading': 185.62326049804688, 'Health': 8, 'Ammo': 10}}, 'items': {-332: {'Id': -332, 'Name': '', 'Type': 'AmmoPickup', 'X': -13.373024940490723, 'Y': -83.5798568725586, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}}}
{'me': 

{'me': {'Id': -2234, 'Name': 'bruh0', 'Type': 'Tank', 'X': 70.74244689941406, 'Y': -7.430698394775391, 'Heading': 99.79598999023438, 'TurretHeading': 99.79598999023438, 'Health': 9, 'Ammo': 10}, 'enemies': {'ManualTank': {'Id': -344, 'Name': 'ManualTank', 'Type': 'Tank', 'X': 51.904788970947266, 'Y': 44.14523696899414, 'Heading': 7.673296928405762, 'TurretHeading': 7.673288345336914, 'Health': -1, 'Ammo': 10}, 'AITank1': {'Id': -512, 'Name': 'AITank1', 'Type': 'Tank', 'X': 57.81350326538086, 'Y': 22.36248779296875, 'Heading': 61.344879150390625, 'TurretHeading': 61.48875045776367, 'Health': 8, 'Ammo': 10}, 'AITank2': {'Id': -680, 'Name': 'AITank2', 'Type': 'Tank', 'X': 45.51226043701172, 'Y': 69.1652603149414, 'Heading': 185.48715209960938, 'TurretHeading': 185.62326049804688, 'Health': 8, 'Ammo': 10}}, 'items': {-332: {'Id': -332, 'Name': '', 'Type': 'AmmoPickup', 'X': -13.373024940490723, 'Y': -83.5798568725586, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}}}
{'me': {

{'me': {'Id': -2538, 'Name': 'bruh1', 'Type': 'Tank', 'X': -0.8575145602226257, 'Y': 57.11725997924805, 'Heading': 151.00027465820312, 'TurretHeading': 151.0002899169922, 'Health': 10, 'Ammo': 10}, 'enemies': {}, 'items': {-176: {'Id': -176, 'Name': '', 'Type': 'AmmoPickup', 'X': -31.140594482421875, 'Y': 35.385860443115234, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}, -308: {'Id': -308, 'Name': '', 'Type': 'AmmoPickup', 'X': -55.5867919921875, 'Y': 35.765846252441406, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}}}
{'me': {'Id': -2538, 'Name': 'bruh1', 'Type': 'Tank', 'X': -0.8575145602226257, 'Y': 57.11725997924805, 'Heading': 151.00027465820312, 'TurretHeading': 151.0002899169922, 'Health': 10, 'Ammo': 10}, 'enemies': {}, 'items': {-176: {'Id': -176, 'Name': '', 'Type': 'AmmoPickup', 'X': -31.140594482421875, 'Y': 35.385860443115234, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}, -308: {'Id': -308, 'Name': '', 'Type': 'AmmoPickup', '

{'me': {'Id': -2538, 'Name': 'bruh1', 'Type': 'Tank', 'X': -0.8575145602226257, 'Y': 57.11725997924805, 'Heading': 151.00027465820312, 'TurretHeading': 151.0002899169922, 'Health': 10, 'Ammo': 10}, 'enemies': {}, 'items': {-176: {'Id': -176, 'Name': '', 'Type': 'AmmoPickup', 'X': -31.140594482421875, 'Y': 35.385860443115234, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}, -308: {'Id': -308, 'Name': '', 'Type': 'AmmoPickup', 'X': -55.5867919921875, 'Y': 35.765846252441406, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}}}
{'me': {'Id': -2538, 'Name': 'bruh1', 'Type': 'Tank', 'X': -0.8575145602226257, 'Y': 57.11725997924805, 'Heading': 151.00027465820312, 'TurretHeading': 151.0002899169922, 'Health': 10, 'Ammo': 10}, 'enemies': {}, 'items': {-176: {'Id': -176, 'Name': '', 'Type': 'AmmoPickup', 'X': -31.140594482421875, 'Y': 35.385860443115234, 'Heading': 0.0, 'TurretHeading': 0.0, 'Health': 0, 'Ammo': 0}, -308: {'Id': -308, 'Name': '', 'Type': 'AmmoPickup', '

In [ ]:
tank_runner.stop()